Ideas:
train hugging face model on debate corpus

tag <--> concatted underlined portion of a paragraph in the card

In [7]:
from Vector_Utils import *
import re

In [8]:
prompt = "Russia loses a nuclear war against the US"
document = Document("article.docx")
full_text = "\n".join([para.text for para in document.paragraphs])

In [9]:
prompt = llmd_prompt(prompt, full_text)

In [10]:
print("Prompt:", prompt)
emphasis_words = prompt.split()

Prompt: U.S. nuclear arsenal's size and accuracy ensures Russia's strategic forces' total destruction in a first-strike scenario.


In [11]:
vectorizer = vectorizer()
prompt_vec = vectorizer.encode(prompt)

In [12]:
paragraphs = document.paragraphs
sentence_enders = re.compile(r'(?<=[.!?])\s+')
para_split = 2

for paragraph in paragraphs:
    highest = 0.4
    best = ()

    sentences = re.split(sentence_enders, paragraph.text)

    if not vectorizer.screen_paragraph(sentences, prompt_vec, threshold=0.4):
        make_small(paragraph)
        continue

    if len(sentences) > para_split:
        print("SPLIT")
        for i in range((len(sentences) // para_split) + 1):
            k = i * para_split
            print(k)
            if k + para_split <= len(sentences):
                sub_paragraph = " ".join(sentences[k:k+para_split])
            else:
                sub_paragraph = " ".join(sentences[k:])
            print(sub_paragraph)
            # if not vectorizer.screen_paragraph(sub_paragraph, prompt_vec, threshold=0.4):
            #     continue
            sub_highest = 0.4
            sub_best = ()
            clauses = clause_separator(sub_paragraph)
            for combination in greedy_combinations(clauses, round(len(clauses) * 0.8), vectorizer, prompt_vec):
                joined = " ".join(combination)
                vector = vectorizer.encode(joined)
                sim_score = vectorizer.similarity(vector, prompt_vec)
                if sim_score > sub_highest:
                    sub_highest = sim_score
                    sub_best = combination
            best += sub_best
            print(best)
    else:
        clauses = clause_separator(paragraph.text)
        for combination in greedy_combinations(clauses, round(len(clauses) * 0.8), vectorizer, prompt_vec):
            joined = " ".join(combination)
            vector = vectorizer.encode(joined)
            sim_score = vectorizer.similarity(vector, prompt_vec)
            if sim_score > highest:
                highest = sim_score
                best = combination

    print(highest, best)
    underline_best_match_in_paragraph(paragraph, best, emphasis_words)

document.save("article_annotated.docx")

0.7391881346702576
SPLIT
0
Modeling a U.S. nuclear attack on Russia requires (1) data on the likely targets of such an attack; (2) data on the number and capabilities of U.S.
('Modeling a U.S. nuclear attack on Russia requires (1) data on the likely targets of such an attack; (2) data',)
2
weapons; (3) a plan that allocates weapons to targets; and (4) formulas for estimating the likely results of the attack. Two other crucial issues in nuclear targeting are the problem of nuclear “fratricide” and the question of whether Russia would be able to launch a retaliatory attack before U.S.
('Modeling a U.S. nuclear attack on Russia requires (1) data on the likely targets of such an attack; (2) data', 'weapons to targets; and (4) formulas', 'for estimating the likely results of the attack. Two other crucial issues in nuclear targeting are the problem of nuclear “fratricide” and the question of whether Russia would be able to launch a retaliatory attack before U.S.')
4
weapons arrived at their 